In [1]:
import pandas as pd
import numpy as np
from sklearn import neighbors
import matplotlib.pyplot as plt
from pandas.tseries.holiday import USFederalHolidayCalendar as calendarUSFH


# Cargar set de datos

In [2]:
trip = pd.read_csv('data/trip_train.csv')

trip_test = pd.read_csv('data/trip_test.csv')

In [3]:
weather = pd.read_csv('data/weather.csv')

In [4]:
station = pd.read_csv('data/station.csv')

# Modificando set de datos

trip_test['start_station_name'] = trip_test['start_station_name']\
    .apply(lambda x : 'Washington at Kearney' if x == 'Washington at Kearny' else x  )

trip_test['start_station_name'] = trip_test['start_station_name']\
    .apply(lambda x : 'Post at Kearney' if x == 'Post at Kearny' else x  )

In [5]:
otherStation = pd.DataFrame([['Broadway at Main',94107],['San Jose Government Center',95113]]\
                            ,columns=['station','zip_code'])

In [6]:
stationReduced = station.loc[:,['id','name','city']]

def choseZipCode(city):
    if city == 'San Francisco' : return 94107
    elif city == 'San Jose' : return 95113
    elif city == 'Redwood City' : return 94063
    elif city == 'Palo Alto' : return 94301
    elif city == 'Mountain View' : return 94041

stationReduced['zip_code'] = stationReduced.loc[:,'city'].apply(choseZipCode)

stationReduced = stationReduced.drop(labels=['id','city'],axis=1)
stationReduced.rename(columns={'name':'station'},inplace=True)

In [7]:
stationReduced = stationReduced.append(otherStation,ignore_index=True)

In [8]:
trip.rename(columns={'start_station_name':'station'},inplace=True)


trip_test.rename(columns={'start_station_name':'station'},inplace=True)

trip_test = trip_test.drop(labels=['zip_code','end_date','end_station_name','end_station_id','bike_id'],axis=1)

In [9]:
trip = trip.drop(labels=['zip_code','end_date','end_station_name','end_station_id','bike_id'],axis=1)

In [10]:
weather['precipitation_inches'] = weather['precipitation_inches'].apply(lambda x : 0.005 if (x == 'T') else float(x))

In [11]:

weather = weather[weather.precipitation_inches.isnull() == False]

weather = weather.fillna(0)

weather['events'] = weather['events'].apply(lambda x : 'Normal' if x == 0 else x)
weather['events'] = weather['events'].apply(lambda x : 'Rain' if x == 'rain' else x)
weather['date'] = weather['date'].apply(lambda date : pd.to_datetime(date).date())

In [12]:
def splitDateAndTime(dateAndTime):
    splitLine = dateAndTime.split(' ')
    return str(splitDate(splitLine[0]))  + splitTime(splitLine[1])

def splitDate(date):
    lineSplit = date.split('/')
    return str(lineSplit[2]) + str(lineSplit[0] if int(lineSplit[0]) >= 10 else '0' + str(lineSplit[0])) + str(lineSplit[1] if int(lineSplit[1]) >= 10 else '0' + str(lineSplit[1]) )

def splitTime(time):
    lineSplit = time.split(':')
    return str(lineSplit[0] if int(lineSplit[0]) >= 10 else '0' + str(lineSplit[0])) + str(lineSplit[1])


In [13]:
trip['time'] = trip['start_date'].apply(lambda x : int(splitDateAndTime(x)[8:12]) )


trip_test['time'] = trip_test['start_date'].apply(lambda x : int(splitDateAndTime(x)[8:12]) )

In [14]:
trip.loc[:,'start_date'] = trip['start_date'].apply(lambda x : pd.to_datetime(splitDateAndTime(x)))


trip_test.loc[:,'start_date'] = trip_test['start_date'].apply(lambda x : pd.to_datetime(splitDateAndTime(x)))

In [15]:
trip.loc[:,'day_of_week'] = trip['start_date'].apply(lambda x : x.dayofweek )


trip_test.loc[:,'day_of_week'] = trip_test['start_date'].apply(lambda x : x.dayofweek )

In [16]:
trip.loc[:,'month'] = trip['start_date'].apply(lambda x : x.month )


trip_test.loc[:,'month'] = trip_test['start_date'].apply(lambda x : x.month )

In [17]:
trip.loc[:,'year'] = trip['start_date'].apply(lambda x : x.year )


trip_test.loc[:,'year'] = trip_test['start_date'].apply(lambda x : x.year )

In [18]:
trip.loc[:,'dayofyear'] = trip['start_date'].apply(lambda x : x.dayofyear )


trip_test.loc[:,'dayofyear'] = trip_test['start_date'].apply(lambda x : x.dayofyear )

In [19]:
trip.loc[:,'start_date'] = trip['start_date'].apply(lambda x : pd.to_datetime(x).date())


trip_test.loc[:,'start_date'] = trip_test['start_date'].apply(lambda x : pd.to_datetime(x).date())

In [20]:
cal = calendarUSFH()
holidays = cal.holidays(return_name=True,start=pd.to_datetime('20130101'), end=pd.to_datetime('20151231'))
holiday_festive_day = pd.DataFrame(holidays,columns=['holiday']).reset_index()
holiday_festive_day.rename(columns={'index':'start_date'},inplace=True)

In [21]:
holiday_festive_day.loc[:,'start_date'] = holiday_festive_day['start_date'].apply(lambda x : pd.to_datetime(x).date())

In [22]:
trip = pd.merge(holiday_festive_day,trip,on=['start_date'],how='right')


trip_test = pd.merge(holiday_festive_day,trip_test,on=['start_date'],how='right')

trip_test.shape

In [23]:
trip.loc[:,'is_holiday'] = trip.holiday.isnull()


trip_test.loc[:,'is_holiday'] = trip_test.holiday.isnull()

In [24]:
trip.loc[:,'is_holiday'] = trip['is_holiday'].apply(lambda x : not x )


trip_test.loc[:,'is_holiday'] = trip_test['is_holiday'].apply(lambda x : not x )

In [25]:
trip.loc[:,'subscription_type'] = trip['subscription_type'].apply(lambda x : 0 if ('Subscriber' == x) else 1 )


trip_test.loc[:,'subscription_type'] = trip_test['subscription_type'].apply(lambda x : 0 if ('Subscriber' == x) else 1 )

In [26]:
trip.sample()

,start_date,holiday,id,duration,station,start_station_id,subscription_type,time,day_of_week,month,year,dayofyear,is_holiday
537987,2014-07-13,NaN,361826,423,2nd at Folsom,62,0,1833,6,7,2014,194,False


trip_test.sample()

trip_test.shape

In [27]:
tripWithZipCode = pd.merge(trip,stationReduced,on=['station'],how='left')


tripWithZipCode_test = pd.merge(trip_test,stationReduced,on=['station'],how='left')

tripWithZipCode_test.shape

In [28]:
tripWithZipCode.sample()

,start_date,holiday,id,duration,station,start_station_id,subscription_type,time,day_of_week,month,year,dayofyear,is_holiday,zip_code
523853,2014-02-08,NaN,178886,250,San Francisco Caltrain 2 (330 Townsend),69,0,1426,5,2,2014,39,False,94107.0


tripWithZipCode_test.sample()

In [29]:
tripWithZipCode = tripWithZipCode.drop(labels=['holiday'],axis=1).dropna()


tripWithZipCode_test = tripWithZipCode_test.drop(labels=['holiday'],axis=1)

tripWithZipCode_test.shape

In [30]:
tripWithZipCode.rename(columns={'start_date':'date'},inplace=True)

tripWithZipCode_test.rename(columns={'start_date':'date'},inplace=True)

tripWithZipCode_test.shape

In [31]:
tripsWithWeather = pd.merge(tripWithZipCode,weather,on=['date','zip_code'],how='left')

tripsWithWeather_test = pd.merge(tripWithZipCode_test,weather,on=['date','zip_code'],how='left')

reduceWeather = weather.drop(labels=['date','events','zip_code'],axis=1)

for value in reduceWeather.columns :
    tripsWithWeather_test[value] = tripsWithWeather_test[value]\
            .apply(lambda x : tripsWithWeather_test[value].mean() if pd.isnull(x) else x)

In [32]:
tripsWithWeather.rename(columns={'start_station_id':'station_id'},inplace=True)

tripsWithWeather_test.rename(columns={'start_station_id':'station_id'},inplace=True)

tripWithZipCode_test.shape

In [33]:
tripsWithWeather = tripsWithWeather.dropna()

In [34]:
tripsWithWeather = tripsWithWeather[tripsWithWeather['duration'] < 5000]

In [35]:
tripsWithWeather.shape

(527164, 35)

tripsWithWeather_test.shape

test_id = tripsWithWeather_test['id']

In [36]:
test_target = tripsWithWeather['duration'][-200000:]

In [37]:
train_target = tripsWithWeather['duration'][0:tripsWithWeather.shape[0]-200001]

In [38]:
tripsWithWeatherReduced = tripsWithWeather.drop(labels=['duration','date','station','events','id'],axis=1)

tripsWithWeatherReduced_test = tripsWithWeather_test.drop(labels=['date','station','events','id'],axis=1)

In [39]:
train = tripsWithWeatherReduced[0:tripsWithWeather.shape[0]-200001]

test = tripsWithWeatherReduced_test

In [40]:
test = tripsWithWeatherReduced[-200000:]

# Reduccion de dimensiones 

In [41]:
from sklearn.decomposition import IncrementalPCA

In [42]:
pca = IncrementalPCA(n_components=8)

In [43]:
tripTransform = pca.fit_transform(tripsWithWeatherReduced)

tripTransform_test = svd_decomposition.fit_transform(tripsWithWeatherReduced_test)

In [44]:
tripTransform.shape

(527164, 8)

tripTransform_test.shape

In [45]:
tripTransformDF = pd.DataFrame(tripTransform)


tripTransformDF_test = pd.DataFrame(tripTransform_test)

In [46]:
tripTransformDF = tripTransformDF.abs()


tripTransformDF_test = tripTransformDF_test.abs()

In [47]:
train = tripTransformDF[0:tripsWithWeather.shape[0]-200001]

In [48]:
test = tripTransformDF[-200000:]

test = tripTransformDF_test

In [49]:

train.shape

(327163, 8)

In [50]:
train_target.shape

(327163,)

# Machine Learning

In [51]:
from sklearn import ensemble

In [52]:
rfr = ensemble.RandomForestRegressor(n_jobs=-1,n_estimators=100,criterion='mse',oob_score=True)

In [53]:
rfr.fit(train,train_target)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=-1, oob_score=True, random_state=None,
           verbose=0, warm_start=False)

In [54]:
test_prediction = rfr.predict(test)

### Conversion para entrega

predictionDF = pd.DataFrame(test_prediction,columns={"duration"})

predictionDF.loc[:,'id'] = test_id.values

predictionDF.to_csv(path_or_buf='prediction',sep=',',header=True,columns=['id','duration'],index=False)

### CrossValidation
#### En caso de querer usar parte del set de entrenamiento como set de prueba 

In [55]:
rfr.score(test,test_target)

-0.013269098613297947

In [56]:
from sklearn.metrics import mean_squared_error

In [57]:
mean_squared_error(test_target,test_prediction)

321226.50452693086

En caso de ser necesario de Cross Validation

In [58]:
predictionDF = pd.DataFrame(test_prediction,columns={"prediction"})

In [59]:
predictionDF.loc[:,'spectate'] = test_target.values

In [60]:
def predictError(predict,spectate):
    if (spectate > predict) : return  ((spectate-predict)/spectate)*100
    else  :  return ((predict-spectate)/predict )*100

In [61]:
predictionDF['errorValue'] = predictionDF.apply(axis=1,func=lambda x : predictError(x[0],x[1]))

In [62]:
predictionDF.sample(5)

,prediction,spectate,errorValue
198102,638.760000,452,29.237898
150400,587.440000,4907,88.028531
26409,704.075333,372,47.164745
177793,590.760000,366,38.045907
146754,614.586476,186,69.735748


In [63]:
print 'El error promedio que comete Random Forest Regresion al predecir es de', float(predictionDF['errorValue'].mean()),'%'

El error promedio que comete Random Forest Regresion al predecir es de 35.9653944619 %


In [64]:
print "El algoritmo Random Forest Regresion predice un ",\
float(predictionDF[predictionDF['errorValue'] < 15].shape[0]) / float(predictionDF.shape[0]) *100 ,\
"% de los valores, con un error menor al 15%"


El algoritmo Random Forest Regresion predice un  21.427 % de los valores, con un error menor al 15%


In [65]:
predictionDF.describe()

,prediction,spectate,errorValue
count,200000.000000,200000.000000,200000.000000
mean,668.620119,655.680590,35.965394
std,186.887641,563.046642,22.262066
min,144.420000,60.000000,0.000000
25%,555.586500,345.000000,17.340117
50%,631.536167,519.000000,33.964092
75%,739.921875,760.000000,52.819906
max,3121.136667,4999.000000,96.293359
